# Plot the variance as a function of the autocorrelation
Compare different estimators of the variance (Gaussian, Gaussian + autocorrelation, Non-Gaussian iid, Non-Gaussian + autocorrelation) with a simulation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
from tqdm.auto import tqdm
from functions import sharpe_ratio_variance, generate_autocorrelated_non_gaussian_data

In [ ]:
import ray
ray.init()

In [ ]:
def legend_thick(ax, *args, **kwargs):
    leg = ax.legend(*args, **kwargs)
    for i in leg.legend_handles:
        i.set_linewidth(7)
        i.set_solid_capstyle('butt')

def remove_empty_axes(axs: np.ndarray) -> None:
    for ax in axs.flatten():
        if (not ax.lines) and (not ax.collections) and (not ax.has_data()):
            ax.axis('off')

In [ ]:
SR0_list = [0, .15, .30, .45, .60]

In [ ]:
mu     = .036
sigma  = .079
T      = 24
gamma3 = -2.448
gamma4 = 10.164
rhos = np.linspace(0, .5, 100)          

fig, axs = plt.subplots(2,3, figsize=(12,8), layout='constrained')
for SR, ax in zip(SR0_list, axs.flatten()):
    SR = SR/np.sqrt(12)
    variances1 = [ sharpe_ratio_variance( SR, T, K=1 ) for rho in rhos ]
    variances2 = [ sharpe_ratio_variance( SR, T, gamma3=gamma3, gamma4=gamma4, K=1 ) for rho in rhos ]
    variances3 = [ sharpe_ratio_variance( SR, T, rho=rho, K=1 ) for rho in rhos ]
    variances4 = [ sharpe_ratio_variance( SR, T, gamma3=gamma3, gamma4=gamma4, rho=rho, K=1 ) for rho in rhos ]
    ax.plot(rhos, variances1, label='Gaussian')
    ax.plot(rhos, variances3, label='Gaussian + autocorrelation')
    ax.plot(rhos, variances2, label='Non-Gaussian iid')
    ax.plot(rhos, variances4, label='Non-Gaussian + autocorrelation')
    ax.axhline( 0, color='black', linewidth=1 )
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles[::-1], labels[::-1])
    ax.set_xlabel('Autocorrelation')
    ax.set_ylabel('Variance')
    ax.set_title(f'Variance of the Sharpe Ratio (SR={SR:.2f})')
remove_empty_axes(axs)
plt.show()

In [ ]:
variances = []
for SR in SR0_list:
    SR = SR/np.sqrt(12)
    for rho in tqdm(rhos):
        X = generate_autocorrelated_non_gaussian_data( T, 10_000, SR0 = SR, name = "severe", rho = rho )
        gamma3 = scipy.stats.skew(X.flatten())                    # Skewness
        gamma4 = scipy.stats.kurtosis(X.flatten(), fisher=False)  # Kurtosis (not excess kurtosis)
        T = X.shape[0]
        variances.append( { 
            'SR': SR,
            'rho': rho,
            'simulation': np.var( X.mean(axis=0) / X.std(axis=0) ),
            'Gaussian': sharpe_ratio_variance( SR, T ),
            'Gaussian + autocorrelation': sharpe_ratio_variance( SR, T, rho=rho ),
            'Non-Gaussian iid': sharpe_ratio_variance( SR, T, gamma3=gamma3, gamma4=gamma4 ),
            'Non-Gaussian + autocorrelation': sharpe_ratio_variance( SR, T, gamma3=gamma3, gamma4=gamma4, rho=rho ),
        } )
variances = pd.DataFrame(variances)

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12, 6), layout='constrained')
for i, SR in enumerate(variances['SR'].unique()):
    ax = axs.flatten()[i]
    tmp = variances[variances['SR'] == SR]
    for column in tmp.columns[2:]:
        ax.plot(tmp['rho'], tmp[column], label=column)
    ax.axhline( 0, linewidth = 1, color = 'black', linestyle = ':' )
    ax.set_ylim( 0, variances.iloc[:,2:].max().max() * 1.05 )
    if i == 0: 
        legend_thick(ax)
    ax.set_xlabel('Autocorrelation')
    ax.set_ylabel('Variance')
    ax.set_title(f'SR={SR:.2f}')
remove_empty_axes(axs)
fig.suptitle( f"Variance of the Sharpe ratio (T={T})")
plt.show()


In [ ]:
@ray.remote
def f3(SR, rho, T, name):
    X = generate_autocorrelated_non_gaussian_data( T, 10_000, SR0 = SR, name = name, rho = rho )
    gamma3 = scipy.stats.skew(X.flatten())                    # Skewness
    gamma4 = scipy.stats.kurtosis(X.flatten(), fisher=False)  # Kurtosis (not excess kurtosis)
    return {
        'name': name,
        'SR': SR,
        'rho': rho,
        'simulation': np.var( X.mean(axis=0) / X.std(axis=0) ),
        'Gaussian iid': sharpe_ratio_variance( SR, T ),
        'Gaussian + autocorrelation': sharpe_ratio_variance( SR, T, rho=rho ),
        'Non-Gaussian iid': sharpe_ratio_variance( SR, T, gamma3=gamma3, gamma4=gamma4 ),
        'Non-Gaussian + autocorrelation': sharpe_ratio_variance( SR, T, gamma3=gamma3, gamma4=gamma4, rho=rho ),
    }

YEARS = 5
PERIODS_PER_YEAR = 12  # 252

T = YEARS * PERIODS_PER_YEAR
rhos = np.linspace(0, .5, 100)
variances = [ 
    f3.remote(SR, rho, T, name) 
    for SR in SR0_list
    for rho in rhos
    for name in ['gaussian', 'mild', 'moderate', 'severe']
]
variances = [ ray.get(v) for v in tqdm(variances) ]
variances = pd.DataFrame(variances)

In [ ]:
for name in variances['name'].unique():
        
    fig, axs = plt.subplots( 2, 3, figsize=(12, 6), layout='constrained', dpi = 300 )
    for i, SR in enumerate(variances['SR'].unique()):
        ax = axs.flatten()[i]
        i1 = variances['SR'] == SR
        i2 = variances['name'] == name
        tmp = variances[ i1 & i2 ]
        for column in tmp.columns[3:]:
            ax.plot(tmp['rho'], tmp[column], label=column)
        ax.axhline( 0, linewidth = 1, color = 'black', linestyle = ':' )
        ax.set_ylim( 0, variances[i2].iloc[:,3:].max().max() * 1.05 )
        if i == 0: 
            #legend_thick(ax)
            handles, labels = ax.get_legend_handles_labels()
        ax.set_xlabel('Autocorrelation')
        ax.set_ylabel('Variance')
        ax.set_title(f'SR={SR:.2f}')
    remove_empty_axes(axs)
    
    ax = axs.flatten()[-1]
    legend_thick( ax, handles, labels, loc = 'center' )

    fig.suptitle( f"Variance of the Sharpe ratio (distribution={name}, T={T})")
    plt.show()


In [ ]:
# Standard deviation

if False: 
        
    for name in variances['name'].unique():
            
        fig, axs = plt.subplots( 2, 3, figsize=(12, 6), layout='constrained', dpi = 300 )
        for i, SR in enumerate(variances['SR'].unique()):
            ax = axs.flatten()[i]
            i1 = variances['SR'] == SR
            i2 = variances['name'] == name
            tmp = variances[ i1 & i2 ]
            for column in tmp.columns[3:]:
                ax.plot(tmp['rho'], np.sqrt(tmp[column]), label=column)
            #ax.axhline( 0, linewidth = 1, color = 'black', linestyle = ':' )
            ax.set_ylim( 0, np.sqrt( variances[i2].iloc[:,3:].max().max() ) * 1.05 )
            if i == 0: 
                #legend_thick(ax)
                handles, labels = ax.get_legend_handles_labels()
            ax.set_xlabel('Autocorrelation')
            ax.set_ylabel('Standard Deviation')
            ax.set_title(f'SR={SR:.2f}')
        remove_empty_axes(axs)
        
        ax = axs.flatten()[-1]
        legend_thick( ax, handles, labels, loc = 'center' )

        fig.suptitle( f"Standard deviation of the Sharpe ratio (distribution={name}, T={T})")
        plt.show()
